# 10 Cross-Validation Techniques For You To Create Models That People Trust
## TODO
![](images/pexels.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@zen-chung?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Zen Chung</a>
        on 
        <a href='https://www.pexels.com/photo/crop-man-playing-with-dog-on-street-5749795/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels.</a> All images are by the author unless specified otherwise.
    </strong>
</figcaption>